# 0. 드라이브 연결

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. 경로 지정

In [7]:
import os
import json

input_folder= "/content/drive/MyDrive/DILAB/YB/GPT_FineTuning/news_data"
output_file = "/content/drive/MyDrive/DILAB/YB/GPT_FineTuning/GPT-3.5-turbo/finetune_data.jsonl"

# 2. 변환 함수 정의

In [9]:
import json

def convert_record_with_fixed_question(record):
    """
    기존 record: {"title": "...", "body": "..."}
    변환 후: {"messages": [{"role":"user",...}, {"role":"assistant",...}]}
    """
    return {
        "messages": [
            {"role": "user", "content": "최근 뉴스를 알려줘."},
            {"role": "assistant", "content": record.get("body", "").strip()}
        ]
    }


# 3. 데이터폼 변환

In [11]:
import os

with open(output_file, "w", encoding="utf-8") as fout:
    for filename in os.listdir(input_folder):
        if filename.endswith(".jsonl"):
            file_path = os.path.join(input_folder, filename)
            with open(file_path, "r", encoding="utf-8") as fin:
                for line in fin:
                    if not line.strip():
                        continue
                    record = json.loads(line)
                    new_record = convert_record_with_fixed_question(record)
                    fout.write(json.dumps(new_record, ensure_ascii=False) + "\n")

print(f"변환 완료: {output_file}")


변환 완료: /content/drive/MyDrive/DILAB/YB/GPT_FineTuning/GPT-3.5-turbo/finetune_data.jsonl
